## Pré-Processamento de Dados

In [2]:
import pandas as pd
import json
import sys
sys.path.append("lib")
import databaseinsert

In [3]:
df = pd.read_json("./datas/tweet.json", lines = True)
df.head(5)

,created_at,likes_count,screen_name,tweet
0,2021-12-16 23:23:06,186,bbcbrasil,"Para especialistas, suicídios de donas de casa..."
1,2021-12-14 15:30:17,48183,CrimesReais,"Assédio, violência, envenenamento, drogas ilíc..."
2,2021-12-17 14:22:38,7,mdhbrasil,"Promover a autonomia financeira das mulheres, ..."
3,2021-12-17 18:55:07,3,policiacivilrs,"Em Bagé, homem de 43 anos é preso, preventivam..."
4,2021-12-17 18:50:19,0,nochldlefbehind,"@aychawane eu achei uma porcaria, real. projet..."


In [4]:
#pip install nltk

In [5]:
# NLKT - Natural Language Toolkit
import nltk
#nltk.download()

In [6]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer

In [7]:
# Funções de pré-processamento (limpeza de dados)
tweet_tokenizer = TweetTokenizer()

def DataCleansing(instance):
  # Remove caracteres, artificios indesejados (links, virgula, pontuação, etc...)
  instance = re.sub(r"http\S+", "", instance).lower().replace('.', '').replace(';', '').replace('-', '').replace(':', '').replace(')', '')
  stopwords = set(nltk.corpus.stopwords.words('portuguese'))
  words = [i for i in instance.split() if not i in stopwords]
  return (" ".join(words))

def PreprocessTweet(tweet):
  tweet = BeautifulSoup(tweet, "html.parser").get_text()
  tweet = re.sub(r"[^a-zA-Zà-úÀ-Ú0-9]+", " ", tweet.lower())
  return tweet

In [8]:
# Realizando a limpeza dos dados
tweets = [DataCleansing(i) for i in df.tweet]

In [9]:
df['clean_data'] = tweets

In [10]:
df['preprocessed_data'] = [PreprocessTweet(tweet) for tweet in df.clean_data]

## Processamento de Dataframe
### - Contagem de quantidade de Palavras

  Identificar as palavras que mais são usadas ou são associadas quando o tema é violência doméstica assédio e feminicídio.

In [11]:
# Importação de biblioteca para contar o número de vezes que uma palavra ocorre
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
cv = CountVectorizer()
counts = cv.fit_transform(df.preprocessed_data)

In [13]:
# Captura as palavras de todos os tweets
cv.get_feature_names()

['001',
 '0genos',
 '12',
 '124',
 '14',
 '144',
 '148',
 '19',
 '1slappy',
 '20',
 '202',
 '2020',
 '2021',
 '24',
 '25',
 '26',
 '290',
 '43',
 '58',
 '800',
 '90daysfiance',
 '90diasparacasar',
 'abalada',
 'abuso',
 'abusos',
 'acaba',
 'acabam',
 'acertando',
 'achei',
 'acompanha',
 'acompanhar',
 'acontecendo',
 'acredito',
 'acusado',
 'acusados',
 'acusação',
 'adam',
 'administrativo',
 'admitido',
 'adorava',
 'adzingamesgay',
 'afastados',
 'afastamento',
 'afazenda',
 'afirma',
 'agir',
 'agora',
 'agredido',
 'agressor',
 'agressores',
 'agressão',
 'agressões',
 'aguentar',
 'ainda',
 'ajuda',
 'ajudar',
 'alegações',
 'alexilapas',
 'alfa',
 'alguém',
 'alternativas',
 'alvoroço',
 'além',
 'amaral',
 'amber',
 'ameaça',
 'ameaças',
 'amo',
 'amor',
 'amoroso',
 'and',
 'ano',
 'anos',
 'antigo',
 'aoyama',
 'apaixonando',
 'apareceu',
 'aplauda',
 'aplicadas',
 'apoiamos',
 'apoio',
 'aposto',
 'aprendir',
 'aproveita',
 'aproveitamos',
 'aproximar',
 'aqula',
 'aqulas

In [14]:
counts.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [15]:
word_count = pd.DataFrame(cv.get_feature_names(), columns = ['word'])
word_count['count'] = counts.sum(axis=0).tolist()[0]
word_count = word_count.sort_values('count', ascending = False).reset_index(drop = True)

In [16]:
words = [index for index, row in word_count.iterrows() if row['word'] == 'violência' or row['word'] == 'doméstica' or row['word'] == 'assédio' or row['word'] == 'feminicídio']
df_wordcount = word_count.drop(words)
df_wordcount.reset_index(inplace=True, drop=True)
df_wordcount.head(5)

,word,count
0,mulheres,18
1,pra,16
2,anos,16
3,contra,15
4,day,15


In [17]:
word_ranking = df_wordcount.set_index('word').T.to_dict('list')
word_ranking

{'mulheres': [18],
 'pra': [16],
 'anos': [16],
 'contra': [15],
 'day': [15],
 'vítima': [15],
 'cara': [9],
 'hoje': [9],
 'mulher': [9],
 'caso': [9],
 'quase': [8],
 'justiça': [8],
 'pq': [8],
 'francisco': [8],
 'polícia': [8],
 'sobre': [7],
 'preso': [7],
 'papa': [7],
 'domestica': [7],
 'gente': [7],
 'boicotar': [6],
 'mello': [6],
 'civil': [6],
 'elogio': [6],
 'âmbito': [6],
 'durante': [6],
 'lindain10572488': [6],
 'trabalho': [6],
 'violencia': [6],
 'nego': [6],
 'brothers': [6],
 'homens': [6],
 'defender': [6],
 'fortaleza': [6],
 'satânica': [6],
 'warner': [6],
 'dois': [6],
 'cantor': [6],
 'fazenda': [6],
 'flagrante': [6],
 'fez': [6],
 'por': [6],
 'deve': [6],
 'todas': [5],
 '25': [5],
 'novo': [5],
 '20': [5],
 'coisa': [5],
 'existem': [5],
 'enquanto': [5],
 'tipo': [5],
 'nada': [5],
 'número': [4],
 'queixa': [4],
 'dia': [4],
 'direitos': [4],
 'diz': [4],
 'machismo': [4],
 'defesa': [4],
 'artistas': [4],
 'clássico': [4],
 'nome': [4],
 'humanos': [

In [18]:
databaseinsert.connect_firebase()

for index, row in df_wordcount.iterrows():
  if index + 1 > 110:
    break
  elif index + 1 <= 110:
    print(index + 1, row['word'], row['count'])

    data = {"word": row['word'],
                  "count": row['count']
            }

    databaseinsert.insert_data(data, 'word_ranking', index + 1)

1 mulheres 18
2 pra 16
3 anos 16
4 contra 15
5 day 15
6 vítima 15
7 cara 9
8 hoje 9
9 mulher 9
10 caso 9
11 quase 8
12 justiça 8
13 pq 8
14 francisco 8
15 polícia 8
16 sobre 7
17 preso 7
18 papa 7
19 domestica 7
20 gente 7
21 boicotar 6
22 mello 6
23 civil 6
24 elogio 6
25 âmbito 6
26 durante 6
27 lindain10572488 6
28 trabalho 6
29 violencia 6
30 nego 6
31 brothers 6
32 homens 6
33 defender 6
34 fortaleza 6
35 satânica 6
36 warner 6
37 dois 6
38 cantor 6
39 fazenda 6
40 flagrante 6
41 fez 6
42 por 6
43 deve 6
44 todas 5
45 25 5
46 novo 5
47 20 5
48 coisa 5
49 existem 5
50 enquanto 5
51 tipo 5
52 nada 5
53 número 4
54 queixa 4
55 dia 4
56 direitos 4
57 diz 4
58 machismo 4
59 defesa 4
60 artistas 4
61 clássico 4
62 nome 4
63 humanos 4
64 tomar 4
65 ninguém 4
66 denunciar 4
67 homem 4
68 posição 4
69 desembargador 4
70 psicológica 4
71 falar 4
72 exmulher 4
73 exmarido 4
74 proteção 4
75 fazer 4
76 meses 4
77 ainda 4
78 famoso 4
79 próprios 4
80 ler 4
81 outra 4
82 combate 4
83 ser 4
84 m